# DS-поток, весна 2025
## Задание ADL.1


**Правила:**

* Дедлайны см. в боте. После дедлайна работы не принимаются кроме случаев наличия уважительной причины.
* Прислать нужно **ноутбук в формате `ipynb`**.
* Следите за размером файлов. **Бот не может принимать файлы весом более 20 Мб.** Если файл получается больше, заранее разделите его на несколько.
* Выполнять задание необходимо полностью самостоятельно. **При обнаружении списывания все участники списывания будут сдавать устный зачет.**
* Решения, размещенные на каких-либо интернет-ресурсах, не принимаются. Кроме того, публикация решения в открытом доступе может быть приравнена к предоставлении возможности списать.
* Для выполнения задания используйте этот ноутбук в качестве основы, ничего не удаляя из него. Можно добавлять необходимое количество ячеек.
* Комментарииь к решению пишите в markdown-ячейках.
* Выполнение задания (ход решения, выводы и пр.) должно быть осуществлено на русском языке.
* Если код будет не понятен проверяющему, оценка может быть снижена.
* Никакой код из данного задания при проверке запускаться не будет. *Если код студента не выполнен, недописан и т.д., то он не оценивается.*
* В каждой задаче не забывайте делать **пояснения и выводы**.


**Баллы за задание**  
Задача 1 &mdash; **90 баллов**  
Задача 2 &mdash; **30 баллов**

Всего &mdash; **120 баллов**  

In [ ]:
# Bot check

# HW_ID: ds_adl1
# Бот проверит этот ID и предупредит, если случайно сдать что-то не то.

# Status: not final
# Перед отправкой в финальном решении удали "not" в строчке выше.
# Так бот проверит, что ты отправляешь финальную версию, а не промежуточную
# Никакие значения в этой ячейке не влияют на факт сдачи работы.

In [1]:
import numpy as np
import pandas as pd
from PIL import Image
from pathlib import Path
from tqdm.auto import tqdm
from functools import lru_cache
import matplotlib.pyplot as plt
from torch.utils.data import Dataset

## Задача 1 &mdash; обучение моделей на разном объеме данных

В этой задаче нужно определить зависимость качества модели от размера обучающих данных и сравнить между собой сверточную и трансформерную модель.

В качестве данных будем использовать [датасет](https://www.kaggle.com/datasets/paramaggarwal/fashion-product-images-dataset) с фотографиями одежды и аксессуаров. В качестве лейблов классификации используем поле `subCategory` в `styles`.

### 1. Подготовка данных



In [ ]:
# Если нужно распаковать zip
import zipfile
zip_file_path = <...>
extract_to_directory = '.'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    file_list = zip_ref.namelist()
    for file in tqdm(file_list, desc="Extracting files", unit="files"):
        zip_ref.extract(file, extract_to_directory)

Для начала определим пути до изображений и таблицы с описанием товаров.

In [7]:
root = Path('fashion-dataset')
images_root = root / 'images'
styles_path = root / 'styles.csv'

Считаем таблицу с описанием товаров. Заметим, что нас интересует только одна колонка &mdash; `subCategory`.

In [ ]:
styles_df = pd.read_csv(styles_path, usecols=['id', 'subCategory'], index_col='id')
styles_df.head()

Заметим, что количество изображений близко, но не точно равно количеству товаров в таблице с описанием.

In [ ]:
len(list(images_root.iterdir())), len(styles_df)

При этом идентификаторы изображений уникальны, а среди идентификаторов в таблице с описанием есть несколько повторений.

In [ ]:
ids1 = [int(path.name.split('.')[0]) for path in images_root.iterdir()]
ids2 = styles_df.index.tolist()
print(len(set(ids1) & set(ids2)) == len(ids1))

Посчитаем количество различных лейблов.

In [ ]:
labels = styles_df.loc[ids1]['subCategory'].values
val, cnt = np.unique(labels, return_counts=True)
cnt

Отсечем все классы, которые встречаются слишком редко &mdash; меньше 100 экземпляров. Для них оставим соответствующие идентификаторы. Визуализируем некоторые картинки.

In [ ]:
labels_unique = val[cnt >= 100]
len(labels_unique)

fig, axs = plt.subplots(27, 9, figsize=(18, 54))
for i, label in enumerate(labels_unique):
    ids = styles_df[styles_df['subCategory'] == label].index[:9]
    for j, id in enumerate(ids):
        image = Image.open(images_root / f'{id}.jpg')
        axs[i, j].imshow(image)
        axs[i, j].set_title(f"{label}, {i}")
        axs[i, j].axis('off')

Можно заметить, что есть классы товаров, которые могут включать в себя другие классы товаров.

In [ ]:
labels = styles_df["subCategory"][styles_df["subCategory"].isin(labels_unique)]
val, cnt = np.unique(labels, return_counts=True)
for v, c in zip(val, cnt):
    print(v, c)

Категория `Accessories` довольно общая, но при этом немногочисленная, ее можно исключить. То же касается категории `Free Gifts`. Категории `Flip Flops` и `Sandal` значительно пересекаются друг с другом, поэтому их лучше объединить.

In [ ]:
labels_unique = labels_unique[~np.isin(labels_unique, ["Accessories",  "Free Gifts", "Flip Flops"])]
styles_df[styles_df["subCategory"] == "Flip Flops"] = "Sandal"
styles_df = styles_df[styles_df["subCategory"].isin(labels_unique)]
num_classes, num_ids = len(labels_unique), len(styles_df.index)
num_classes, num_ids

Составим карту отображения лейблов в классы.

In [ ]:
label2class = {label: i for i, label in enumerate(labels_unique)}
label2class

Проверим, для всех ли идентификаторов из таблицы стилей есть соответствующие изображения. Удалим идентификаторы, для которых изображения не нашлись.

In [ ]:
not_found_ids = []
for id in tqdm(styles_df.index):
    if not (images_root / f'{id}.jpg').exists():
        not_found_ids.append(id)
styles_df = styles_df[~styles_df.index.isin(not_found_ids)]

# Проверим, что в таблице остались только уникальные идентификаторы.
assert len(styles_df.index) == len(np.unique(styles_df.index))

Реализуем класс для работы с данными.

In [26]:
class StylesDataset(Dataset):
    def __init__(self, images_root, styles_df, label2class, ids, transform):
        self.images_root = images_root
        self.styles_df = styles_df
        self.label2class = label2class
        self.ids = ids
        self.transform = transform

    def __len__(self):
        return len(self.ids)

    @lru_cache(maxsize=400)
    def __getitem__(self, i):
        id = self.ids[i]
        image = Image.open(self.images_root / f'{id}.jpg')
        image = self.transform(image)
        label = self.styles_df.loc[id, 'subCategory']
        cls = self.label2class[label]
        return image, cls

### 2. Обучение моделей

Модели можно выбрать из [timm](https://github.com/huggingface/pytorch-image-models), [torchvision](https://pytorch.org/vision/stable/models.html) или [huggingface](https://huggingface.co/docs/transformers/index). В качестве сверточной модели можно использовать EfficientNet или более мощную модель. Более мощную модель будем определять по рейтингу top-1 моделей предобученных на ImageNet. Для сравнения моделей можно воспользоваться [рейтингом paperswithcode](https://paperswithcode.com/sota/image-classification-on-imagenet) или соотвествующими результатами в научных статьях, посвященным моделям для классификации изображений. В качестве трансформерной модели можно использовать ViT, DeiT, CeiT, VOLO, SWIN или более мощные модели. Для того, чтобы достигнуть хорошего качества, используйте уже предобученные модели.

*Задача*  &mdash;  для разных обучающих наборов данных: 2000, 8000 снимков и всего набора данных (42749 снимков) посчитайте weighted accuracy (усредненную accuracy по всем классам) для сверточной и трансформерной моделей. На тест выделите 1000 снимков. На большом объеме данных трансформерная модель не должна быть сильно хуже сверточной. На всем датасете обе модели должны пробить порог weighted accuracy равный 0.8.

Возможно, *для улучшения качества трансформера* вам могут помочь:
- [пример](https://keras.io/examples/vision/vit_small_ds/) с построением трансформерной модели на keras, где описываются аугментации, shifted patches, local attention;
- [статья](https://arxiv.org/pdf/2106.03746.pdf), где используют доп. лосс для обучения произвольной трансформерной модели на небольшом объеме данных.

*Правила проведения экспериментов*.
- Для всех экспериментов используйте одинаковый пайплайн семплирования данных &mdash; аугментации, размер батча, метод выбора данных для формирования батча.
- Для экспериментов с одинаковым количеством обучающих данных используйте одинакове число эпох для обучения. 
- Для каждой модели (сверточной и трансформерной) зафиксируйте пайплайн обучения &mdash; саму модель, оптимизатор, learing rate scheduling, регуляризацию весов модели, лоссы.
- Поясните выбор стратегии семплирования данных и пайпайна обучения.

### 3. Анализ моделей

1. Оцените размеры моделей (MB). Сравните время вычислений (сек.) и затрачиваемую память (MB) на обучении и на инференсе для обеих моделей.


2. Выберите лучшие сверточную и трансформерную модели и проведите анализ ошибок моделей. 
- Определите, в каких классах ошибались модели.
- Визуализируйте примеры с правильными и неправильнымм предсказаниями для каждой модели.
- Предположите, с чем связаны ошибки.

3. Сделайте **выводы** по задаче.

## Задача 2 &mdash; Attention vs. Outlook attention

Реализуйте классы 1 блока Attention и 1 блока Outlook attention.

Примените написанные слои к случайному тензору или произвольной картинке. Сравните:
* время вычислений (сек)
* затрачиваемую память (MB)


Сделайте **выводы** по задаче.